In [ ]:
# If running in a fresh kernel, install missing deps
import sys, subprocess
subprocess.run([sys.executable, "-m", "pip", "install", "SQLAlchemy", "psycopg2-binary", "python-dotenv"], check=False)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import subprocess
import sys

In [ ]:
import sqlalchemy as sa
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Time, Text
import pandas as pd
import os
import psycopg2
from dotenv import load_dotenv

load_dotenv()
pg_user = os.getenv("PG_USER", "root")
pg_password = os.getenv("PG_PASSWORD", "practice")
pg_host = os.getenv("PG_HOST", "127.0.0.1")
pg_port = os.getenv("PG_PORT", "5432")
pg_db = os.getenv("PG_DB", "uberda")

engine = create_engine(f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}", echo=True)

try:
    with engine.connect() as conn:
        result = conn.execute(sa.text("SELECT 1"))
        print("Database connection successful!")
except Exception as e:
    print(f"Database connection failed: {e}")


2025-09-24 02:45:43,171 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-09-24 02:45:43,171 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-24 02:45:43,174 INFO sqlalchemy.engine.Engine select current_schema()
2025-09-24 02:45:43,174 INFO sqlalchemy.engine.Engine [raw sql] {}


2025-09-24 02:45:43,176 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-09-24 02:45:43,177 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-24 02:45:43,178 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-24 02:45:43,179 INFO sqlalchemy.engine.Engine SELECT 1
2025-09-24 02:45:43,179 INFO sqlalchemy.engine.Engine [generated in 0.00105s] {}
Database connection successful!
2025-09-24 02:45:43,180 INFO sqlalchemy.engine.Engine ROLLBACK


In [13]:
metadata = MetaData()

uber_bookings = Table(
    'uber_bookings',
    metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('date', Date, nullable=False),
    Column('time', Time, nullable=False),
    Column('booking_id', String(50), nullable=False, unique=True),
    Column('booking_status', String(50), nullable=False),
    Column('customer_id', String(50), nullable=False),
    Column('vehicle_type', String(50), nullable=False),
    Column('payment_method', String(50), nullable=True)
)

print("Table schema defined:")
print(f"Table name: {uber_bookings.name}")
print("Columns:")
for column in uber_bookings.columns:
    print(f"  - {column.name}: {column.type}")


Table schema defined:
Table name: uber_bookings
Columns:
  - id: INTEGER
  - date: DATE
  - time: TIME
  - booking_id: VARCHAR(50)
  - booking_status: VARCHAR(50)
  - customer_id: VARCHAR(50)
  - vehicle_type: VARCHAR(50)
  - payment_method: VARCHAR(50)


In [14]:
try:
    metadata.create_all(engine)
    print("Table 'uber_bookings' created successfully!")
except Exception as e:
    print(f"Error creating table: {e}")
    if "already exists" in str(e).lower():
        print("Table already exists, continuing...")


2025-09-24 02:45:53,062 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-24 02:45:53,065 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-09-24 02:45:53,066 INFO sqlalchemy.engine.Engine [generated in 0.00080s] {'table_name': 'uber_bookings', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-09-24 02:45:53,072 INFO sqlalchemy.engine.Engine 
CREATE TABLE uber_bookings (
	id SERIAL NOT NULL, 
	date DATE NOT NULL, 
	time TIME WITHOUT TIME ZONE NOT NULL, 
	booking_id VARCHAR(50) NOT NULL, 
	booking_status VARCHAR

In [ ]:
# Read the CSV file from env var with default
from dotenv import load_dotenv
load_dotenv()
csv_path = os.getenv("CSV_PATH", "./data/cleaned_up_pandas.csv")

print(f"Reading CSV file: {csv_path}")
print(f"File exists: {os.path.exists(csv_path)}")

# Read the CSV with proper data types
df = pd.read_csv(csv_path)

print("CSV loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("First few rows:")
print(df.head())


Reading CSV file: ../data/cleaned_up_pandas.csv
File exists: True
CSV loaded successfully!
Shape: (150000, 7)
Columns: ['Date', 'Time', 'Booking ID', 'Booking Status', 'Customer ID', 'Vehicle Type', 'Payment Method']
First few rows:
         Date      Time    Booking ID       Booking Status   Customer ID  \
0  2024-01-01  00:19:34  "CNR4352144"            Completed  "CID8362794"   
1  2024-01-01  01:35:18  "CNR9147645"            Completed  "CID8300238"   
2  2024-01-01  01:37:50  "CNR1009222"  Cancelled by Driver  "CID2030746"   
3  2024-01-01  01:48:03  "CNR2740479"  Cancelled by Driver  "CID3231181"   
4  2024-01-01  01:49:56  "CNR7650148"  Cancelled by Driver  "CID3381661"   

  Vehicle Type Payment Method  
0         bike           Cash  
1      go mini    Uber Wallet  
2     go sedan            NaN  
3         auto            NaN  
4     go sedan            NaN  


In [16]:

df_clean = df.copy()


df_clean['Booking ID'] = df_clean['Booking ID'].str.replace('"', '')
df_clean['Customer ID'] = df_clean['Customer ID'].str.replace('"', '')


df_clean['Date'] = pd.to_datetime(df_clean['Date']).dt.date
df_clean['Time'] = pd.to_datetime(df_clean['Time'], format='%H:%M:%S').dt.time


df_clean['Payment Method'] = df_clean['Payment Method'].fillna('')


df_clean = df_clean.rename(columns={
    'Date': 'date',
    'Time': 'time', 
    'Booking ID': 'booking_id',
    'Booking Status': 'booking_status',
    'Customer ID': 'customer_id',
    'Vehicle Type': 'vehicle_type',
    'Payment Method': 'payment_method'
})

print(df_clean.head())
print(f"Data types:")
print(df_clean.dtypes)


         date      time  booking_id       booking_status customer_id  \
0  2024-01-01  00:19:34  CNR4352144            Completed  CID8362794   
1  2024-01-01  01:35:18  CNR9147645            Completed  CID8300238   
2  2024-01-01  01:37:50  CNR1009222  Cancelled by Driver  CID2030746   
3  2024-01-01  01:48:03  CNR2740479  Cancelled by Driver  CID3231181   
4  2024-01-01  01:49:56  CNR7650148  Cancelled by Driver  CID3381661   

  vehicle_type payment_method  
0         bike           Cash  
1      go mini    Uber Wallet  
2     go sedan                 
3         auto                 
4     go sedan                 
Data types:
date              object
time              object
booking_id        object
booking_status    object
customer_id       object
vehicle_type      object
payment_method    object
dtype: object


In [ ]:
try:
    with engine.connect() as conn:
        # Check if table has any data
        result = conn.execute(sa.text("SELECT COUNT(*) FROM uber_bookings"))
        count = result.scalar()
        print(f"Current records in database: {count}")
        
        # Non-interactive: clear if CLEAR_EXISTING=y in env
        clear_flag = os.getenv("CLEAR_EXISTING", "n").lower() == "y"
        if count > 0 and clear_flag:
            print("Clearing existing data due to CLEAR_EXISTING=y ...")
            conn.execute(sa.text("DELETE FROM uber_bookings"))
            conn.commit()
            print("Existing data cleared!")
        elif count > 0:
            print("Keeping existing data. New data will be inserted (duplicates skipped).")
                
except Exception as e:
    print(f"Error checking database: {e}")


2025-09-24 02:45:53,721 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-24 02:45:53,722 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM uber_bookings
2025-09-24 02:45:53,722 INFO sqlalchemy.engine.Engine [generated in 0.00127s] {}
Current records in database: 0
2025-09-24 02:45:53,725 INFO sqlalchemy.engine.Engine ROLLBACK


In [18]:
# Insert data into PostgreSQL database
print("Inserting data into PostgreSQL database...")

data_to_insert = df_clean.to_dict('records')

batch_size = 1000
total_records = len(data_to_insert)
inserted_count = 0
skipped_count = 0

print(f"Total records to insert: {total_records}")
print(f"Processing in batches of {batch_size}...")

try:
    with engine.connect() as conn:
        for i in range(0, total_records, batch_size):
            batch = data_to_insert[i:i + batch_size]
            
            try:
                conn.execute(
                    uber_bookings.insert().on_conflict_do_nothing(),
                    batch
                )
                conn.commit()
                inserted_count += len(batch)
                
                if (i // batch_size + 1) % 10 == 0:  # Progress every 10 batches
                    print(f"Processed {inserted_count}/{total_records} records...")
                    
            except Exception as e:
                print(f"Batch {i//batch_size + 1} had issues: {e}")
                for record in batch:
                    try:
                        conn.execute(
                            uber_bookings.insert().on_conflict_do_nothing(),
                            record
                        )
                        inserted_count += 1
                    except:
                        skipped_count += 1
                conn.commit()
                
    print(f"Data insertion completed!")
    print(f"Successfully inserted: {inserted_count} records")
    print(f"⏭Skipped (duplicates): {skipped_count} records")
    
except Exception as e:
    print(f"Error during data insertion: {e}")


Inserting data into PostgreSQL database...
Total records to insert: 150000
Processing in batches of 1000...
Batch 1 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 2 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 3 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 4 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 5 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 6 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 7 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 8 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 9 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 10 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 11 had issues: 'Insert' object has no attribute 'on_conflict_do_nothing'
Batch 12 had issues: 'Insert' ob

In [19]:
try:
    with engine.connect() as conn:
        # Get total count
        result = conn.execute(sa.text("SELECT COUNT(*) FROM uber_bookings"))
        total_count = result.scalar()
        print(f"Total records in database: {total_count}")
        
        # Get sample data
        result = conn.execute(sa.text("SELECT * FROM uber_bookings LIMIT 5"))
        sample_data = result.fetchall()
        print(f"Sample data:")
        for row in sample_data:
            print(f"  {row}")
            
        result = conn.execute(sa.text("""
            SELECT 
                booking_status,
                COUNT(*) as count
            FROM uber_bookings 
            GROUP BY booking_status 
            ORDER BY count DESC
        """))
        status_stats = result.fetchall()
        print(f"Booking status distribution:")
        for status, count in status_stats:
            print(f"  {status}: {count}")
            
        result = conn.execute(sa.text("""
            SELECT 
                vehicle_type,
                COUNT(*) as count
            FROM uber_bookings 
            GROUP BY vehicle_type 
            ORDER BY count DESC
        """))
        vehicle_stats = result.fetchall()
        print(f"Vehicle type distribution:")
        for vehicle, count in vehicle_stats:
            print(f"  {vehicle}: {count}")
            
except Exception as e:
    print(f"Error verifying data: {e}")


2025-09-24 02:45:54,540 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-24 02:45:54,541 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM uber_bookings
2025-09-24 02:45:54,542 INFO sqlalchemy.engine.Engine [cached since 0.8205s ago] {}
Total records in database: 0
2025-09-24 02:45:54,543 INFO sqlalchemy.engine.Engine SELECT * FROM uber_bookings LIMIT 5
2025-09-24 02:45:54,543 INFO sqlalchemy.engine.Engine [generated in 0.00045s] {}
Sample data:
2025-09-24 02:45:54,544 INFO sqlalchemy.engine.Engine 
            SELECT 
                booking_status,
                COUNT(*) as count
            FROM uber_bookings 
            GROUP BY booking_status 
            ORDER BY count DESC
        
2025-09-24 02:45:54,545 INFO sqlalchemy.engine.Engine [generated in 0.00040s] {}
Booking status distribution:
2025-09-24 02:45:54,546 INFO sqlalchemy.engine.Engine 
            SELECT 
                vehicle_type,
                COUNT(*) as count
            FROM uber_bookings 
           

In [ ]:
# Postgres upsert insert using SQLAlchemy dialect
from sqlalchemy.dialects.postgresql import insert as pg_insert

batch_size = 1000
rows = df_clean.to_dict('records')

print("Writing with ON CONFLICT DO NOTHING on booking_id...")
written = 0
skipped = 0

with engine.begin() as conn:  # transactional
    for i in range(0, len(rows), batch_size):
        batch = rows[i:i+batch_size]
        stmt = pg_insert(uber_bookings).values(batch)
        stmt = stmt.on_conflict_do_nothing(index_elements=[uber_bookings.c.booking_id])
        res = conn.execute(stmt)
        written += res.rowcount or 0

print(f"done. inserted={written} (skipped duplicates shown by 0 rowcount)")


In [20]:
try:
    with engine.connect() as conn:
        # Create indexes on commonly queried columns
        indexes = [
            "CREATE INDEX IF NOT EXISTS idx_booking_date ON uber_bookings(date)",
            "CREATE INDEX IF NOT EXISTS idx_booking_status ON uber_bookings(booking_status)",
            "CREATE INDEX IF NOT EXISTS idx_vehicle_type ON uber_bookings(vehicle_type)",
            "CREATE INDEX IF NOT EXISTS idx_customer_id ON uber_bookings(customer_id)",
            "CREATE INDEX IF NOT EXISTS idx_payment_method ON uber_bookings(payment_method)",
            "CREATE INDEX IF NOT EXISTS idx_date_time ON uber_bookings(date, time)"
        ]
        
        for index_sql in indexes:
            try:
                conn.execute(sa.text(index_sql))
                print(f"Created index: {index_sql.split()[-1]}")
            except Exception as e:
                print(f"Index creation warning: {e}")
        
        conn.commit()
        print("All indexes created successfully!")
        
except Exception as e:
    print(f"Error creating indexes: {e}")

2025-09-24 02:45:54,553 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-24 02:45:54,554 INFO sqlalchemy.engine.Engine CREATE INDEX IF NOT EXISTS idx_booking_date ON uber_bookings(date)
2025-09-24 02:45:54,555 INFO sqlalchemy.engine.Engine [generated in 0.00121s] {}
Created index: uber_bookings(date)
2025-09-24 02:45:54,557 INFO sqlalchemy.engine.Engine CREATE INDEX IF NOT EXISTS idx_booking_status ON uber_bookings(booking_status)
2025-09-24 02:45:54,558 INFO sqlalchemy.engine.Engine [generated in 0.00050s] {}
Created index: uber_bookings(booking_status)
2025-09-24 02:45:54,561 INFO sqlalchemy.engine.Engine CREATE INDEX IF NOT EXISTS idx_vehicle_type ON uber_bookings(vehicle_type)
2025-09-24 02:45:54,562 INFO sqlalchemy.engine.Engine [generated in 0.00042s] {}
Created index: uber_bookings(vehicle_type)
2025-09-24 02:45:54,564 INFO sqlalchemy.engine.Engine CREATE INDEX IF NOT EXISTS idx_customer_id ON uber_bookings(customer_id)
2025-09-24 02:45:54,564 INFO sqlalchemy.engine.Engine

## Basic Queries
```sql
-- Get total number of bookings
SELECT COUNT(*) FROM uber_bookings;

-- Get bookings by status
SELECT booking_status, COUNT(*) as count 
FROM uber_bookings 
GROUP BY booking_status 
ORDER BY count DESC;

-- Get bookings by vehicle type
SELECT vehicle_type, COUNT(*) as count 
FROM uber_bookings 
GROUP BY vehicle_type 
ORDER BY count DESC;
```

## Time-based Analysis
```sql
-- Bookings per hour of day
SELECT EXTRACT(hour FROM time) as hour, COUNT(*) as bookings
FROM uber_bookings 
GROUP BY hour 
ORDER BY hour;

-- Bookings per day of week
SELECT EXTRACT(dow FROM date) as weekday, COUNT(*) as bookings
FROM uber_bookings 
GROUP BY weekday 
ORDER BY weekday;

-- Bookings per month
SELECT EXTRACT(year FROM date) as year, 
       EXTRACT(month FROM date) as month, 
       COUNT(*) as bookings
FROM uber_bookings 
GROUP BY year, month 
ORDER BY year, month;
```

## Customer Analysis
```sql
-- Top customers by booking count
SELECT customer_id, COUNT(*) as total_bookings
FROM uber_bookings 
GROUP BY customer_id 
ORDER BY total_bookings DESC 
LIMIT 10;

-- Payment method preferences
SELECT payment_method, COUNT(*) as count
FROM uber_bookings 
WHERE payment_method != ''
GROUP BY payment_method 
ORDER BY count DESC;
```
